# Simulated Annealing to find the optimal observing schedule for optical follow-up observations from LIGO

This notebook employs simulated annealing, in order to optimize the observing schedule for (in this case) optical follow-up observations of LIGO GW-data. We are doing the following steps:

- Import all necessary libraries & define some constants
- Define all functions
- start the simulated annealing and determine the optimal schedule

The general idea is to minimize "energy" of a boltzman-statistical energy probability distribution function: $$p(E_j) \propto \exp\left(-\frac{E_j-E_i}{c\cdot T}\right)$$In our case we want to minimize the observing time. We therefore start with an observing plan and iteratively change it ("atomic change", i.e. swap two rows). Within each iteration we calculate the "energy" of the new plan and if the energy and therefore an energy dependant acceptance probability is higher than a random number, we continue with the new observing schedule. By lowering the temperature we find with an increasing probability the minimal energy state, i.e. the best observing plan

## Imports and initial variables

We import the needed packages, set the observatory location, specify the path to the object list and the starting and ending time of the observing night. 

In [1]:
# top-down code to plan observing schedule for a LIGO event

import numpy as np
import pandas as pd

from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time, TimeDelta
from astropy import units as u

from tqdm import tqdm

def conv_to_deg(deg, arcmin, arcsec):
    return (deg+ arcmin/60+ arcsec/3600)*u.deg

# cross-check values from Ananya/Arno/Juliana
Wendelstein_loc = EarthLocation(lat = conv_to_deg(47, 42, 13.1), lon = conv_to_deg(12, 0, 43.4), height = 1838*u.m)

# define average telescope speed in deg/sec
v_dome = 2*u.deg/(1*u.second)
v_tel_az = 3.6*u.deg/(1*u.second)
v_tel_alt = 3.0*u.deg/(1*u.second)


superevent_name = "S230615az"
target_filename = f"PGAL/PGAL_{superevent_name}.ecsv"

starting_time = '2023-06-15T20:49:00'
ending_time = '2023-06-15T22:19:00'

starting_time = Time(starting_time, format = 'isot', scale='utc')
ending_time = Time(ending_time, format = 'isot', scale='utc')

utc_offset = TimeDelta(2*60*60, format = "sec")

starting_time = starting_time + utc_offset
ending_time = ending_time + utc_offset

starting_time_jd = starting_time.jd
ending_time_jd = ending_time.jd

print(f"We have chosen {starting_time} as our starting time of observations and {ending_time} as our ending time.")
print(f"In julian time we have: {starting_time_jd} and {ending_time_jd}")

We have chosen 2023-06-15T22:49:00.000 as our starting time of observations and 2023-06-16T00:19:00.000 as our ending time.
In julian time we have: 2460111.4506944446 and 2460111.5131944446


## Define the basic functions

### Reading a target list

The target list is a .csv file that contains RA, DEC, Sersic fit, redshift and probability of being the host galaxy. With this function one can read this file and make some necessary but simple changes for our further usage: we rename the columns and add SkyCoord objects to the list, which will come in handy later on when calculating the Alt and Az of each object.

In [2]:

def read_target_list(filename):

    """
    Function to read a target list from a .csv file and add the SkyCoord objects as a new column. \n
    It further renames the columns "TARGET_RA" and "TARGET_DEC" to "RA" and "dec"
 
    Parameters
    ----------

    filename: str
        filename is the path specifying where to get the target list as a .csv file from

    Returns
    -------

    file: pd.DataFrame
        target list containing the .csv files information and the added SkyCoords as a pandas data frame object
    """

    # read from a file
    print("Reading csv table...")
    file = pd.read_csv(filename)
    
    # rename columns
    file = file.rename(columns = {"TARGET_RA": "RA", "TARGET_DEC": "dec"})

    # calculate and add SkyCoords for AltAz calculation later
    print("Adding SkyCoords to it...")
    sky_coords = SkyCoord(file["RA"], file["dec"], unit = "deg")

    file["SKY_COORD"] = sky_coords
    
    #return the pandas dataframe (i.e. the unmodified initial target list)
    print("Finished, now returning pandas df")
    return file

target_list = read_target_list(target_filename)

Reading csv table...
Adding SkyCoords to it...
Finished, now returning pandas df


### Initialize the observing plan

We have to start with some plan, so we take out all objects with a declination that Wendelstein can never reach (-20deg and below) and return the target list.

*Note: We initially thought to calculate the Alt and Az for each object, but this is not a useful thing, since the observation time is variable in our algorithm and using a set time therefore does not make any sense.*

In [3]:
def initialize_observing_plan(target_list):

        """"
        Initialize the observing plan, which means (for now simply):
                - delete objects with a declination less or equal to -20 degrees, since Wendelstein can never observe that

        Parameters
        ----------

        target_list: pd.DataFrame
                a target list as a pandas dataframe containing RA, dec, sersic, redshift and probability (and SkyCoords)
        
        Returns
        -------
        target_list: pd.DataFrame
                target list after "clean up"
        

        """

        # delete all entries with dec lower than -20°, since Wendelstein can't observe that
        target_list = target_list[target_list["dec"]> -20]

        # return modified target_list
        return target_list

initial_observing_plan = initialize_observing_plan(target_list)

### Change plan

We define our plan change function, which simply exchanges two rows on a random basis and returns this new plan

In [4]:
def change_plan(plan):

    """
    Randomly picks two rows to be interchanged in the observing plan ("atomic change")

    Parameters:
    plan: pd.DataFrame
        a observation plan that is to be modified

    Returns:
    --------

    pd.DataFrame: plan
        return the modified plan 
    
    """

    # get the plan length to know the range for choosing random numbers as row changes
    plan_len = len(plan)

    # pick the random numbers in the according range
    rnd1, rnd2 = np.random.randint(0,plan_len), np.random.randint(0,plan_len)

    # get the original rows
    row1, row2 = plan.iloc[rnd1].copy(), plan.iloc[rnd2].copy()

    # interchange the rows
    plan.iloc[rnd1], plan.iloc[rnd2] = row2, row1
    
    #return the modified plan
    return plan

### Calculate the slewing time of the telescope

We calculate the angular seperation and from there get the time needed to slew from one object to another

In [5]:
def slewing_time(ra1, dec1, ra2, dec2, time):

    """
    Calculate the slewing time based on its angular seperation between two targets with a specified RA and dec:
    
    Quoting from e-mail (Arno Riffeser, Christoph Riess,...):
    'In the meantime I would calculate the total time, by calculating the time for the dome, 
    then adding the time for the telescope-Az and finally adding the time for the Alt:
    Dome-Az ~= 2.0 deg/sec
    Tel-Az  ~= 3.6 deg/sec
    Tel-Alt ~= 3.0 deg/sec'

    Parameters
    ----------
        ra1: int/float
            right ascension of target one
        dec1: int/float
            declination of target one
        ra2: int/float
            right ascension of target one
        dec2: int/float
            declination of target one
    
    Returns
    -------
        slewing time: float (unit: seconds)
            the time in seconds the telescope needs to move from one target to the second
    """
    """
    # get SkyCoord object and then calculate target ra and dec
    current_SkyCoord = SkyCoord(ra1, dec1, unit = "deg")
    target_SkyCoord = SkyCoord(ra2, dec2, unit = "deg")
    
    target_AltAz = target_SkyCoord.transform_to(AltAz(obstime = time, location = Wendelstein_loc))
    current_AltAz = current_SkyCoord.transform_to(AltAz(obstime = time, location = Wendelstein_loc))
    
    # calculate the distances in degrees...
    delta_alt, delta_az = abs(target_AltAz.alt.to_value()-current_AltAz.alt.to_value())*u.deg, abs(target_AltAz.az.to_value() - current_AltAz.az.to_value())*u.deg
    
    # calculate the times
    t_dome = delta_az/v_dome
    t_tel_alt = delta_alt/v_tel_alt
    t_tel_az = delta_az/v_tel_az
    """
    t_dome = abs(ra2-ra1)*u.deg/v_dome
    t_tel_alt = abs(dec2-dec1)*u.deg/v_tel_alt
    t_tel_az = abs(ra2-ra1)*u.deg/v_tel_az
    
    # add them together
    t_tot = t_dome + t_tel_alt + t_tel_az
    
    # and return the time
    return t_tot

    # turn the ra and dec values from degrees to radians
    # ra1, dec1, ra2, dec2 = np.deg2rad([ra1, dec1, ra2, dec2])
    

    # calculate the angular seperation in degrees
    # NOTE: the max and min check is to ensure that any computaion error leading to 1.00000000001 for example does not result in a math error in the arccos
    # this seems to be a too optimistic way
    """sep_arg = np.sin(dec1)*np.sin(dec2) + np.cos(dec1)*np.cos(dec2)*np.cos(ra1-ra2)
    sep_arg = max(-1, sep_arg)
    sep_arg = min(1, sep_arg)
    angular_seperation = np.rad2deg(np.arccos(sep_arg))*u.deg

    #return the slewing time
    return angular_seperation/v_telescope"""
    
    

### Print the observation plan and the according schedule including alt and az data

In [6]:
def print_observing_plan(plan, schedule):

    """
    print the plan and schedule

    Parameters
    ----------
        plan: pd.DataFrame
            the plan table that includes RA, dec, sersic, redshift, probability (and SkyCoords)
        schedule: dictionary
            the observation schedule including Alt and Az
    """

    print(f"Our observation plan is \n {plan}")
    print()
    print(f"With the following schedule: \n {pd.DataFrame(schedule)}")

### Calculate the required exposure time for each object

In [7]:
# def determine_required_exposure_time(target, starting_time):

#     # what?
    
#     return 50*u.second # + readout time # Ananya's

## Simulated Annealing process

This is divided into two main steps:
- evualuate a plan
- iterate through many plans and lower the "temperature" to accept only energetically preferable "states" (i.e. plans)

### Evaluate a plan

Here we evaluate a plan, which means we iterate through the plans target list/ranking and calculate an energy according to:
$$p*\left(\frac{T_{end}-T_{curr}}{T_{end}-T_{start}}\right)^\alpha$$
- p: the probability of each object determined by Galaxy_Selector_MOC.ipynb
- T_{end}: the ending time of observation for that night
- T_{curr}: the current time in the sim annealing process
- T_{start}: starting time of observation
- \alpha: exponent to increase severance of timing optimization

We include two steps to check for observability:
1. Observed before sun rises?
2. Above horizon?

If it does not pass 1., we simply set an entry in the schedule for "not observed" and continue. In the latter case we do the following: if it is not observable due to the sky position, we just don't add any energy at all. Otherwise, i.e. if its observable by time and position we add the energy as stated above.

Further steps are to update the plan by changing the relevant times and targeting info.

In [8]:
def evaluate_plan(plan, starting_time, ending_time, alpha=4):

    """
    Evaluate a single plan according to energy = sum_i p_i ((t_end - tobs_i)/(t_end - t_begin))^alpha.
    We also include observability checks and "punish" plans that have objects below the horizon.

    Parameters
    ----------
        plan: pd.DataFrame
            the to be evaluated observation plan
        starting_time: Time (from astropy.time)
            starting time for observations of the night
        ending_time: Time (from astropy.time)
            ending time for observations of the night
        alpha (optional): float/int
            exponent to change prioritization of getting things done quickyl
    
    Returns
    -------
        energy: float
            the "energy" this schedule has
        schedule: dict
            the schedule dictionary containing whether an object has been ovserved, start and ending time and AltAz
    """
    
    #print("starting plan evaluation...")

    # set the initial values from the first item in the plan
    current_time = starting_time
    current_RA = plan['RA'][0]
    current_dec = plan['dec'][0]

    energy = 0  # we start with zero energy...
    
    schedule = []
    
    # iterate through each object in the plan
    for index, target in plan.iterrows():

        #print("our current target is: ", target)

        # calculate slewing and exposure time for this target
        # now this isn't perfect, since we don't know the exact slewing time and therefore don't "really" know alt and az
        # however, the error should be very small, therefore we just take the current time
        slew_time  = slewing_time(current_RA, current_dec, target['RA'],target['dec'], current_time)
        exposure_time = 500

        if(current_time + slew_time + exposure_time < ending_time) : # we can observe this target (concerning the timing)

            # create an empty dictionary to store the following values...
            schedule_entry = {}

            # we say we observed this object, calculate the starting time of observation and add the needed exposure time all to the dict
            schedule_entry['observed'] = True
            schedule_entry['starting_time'] = current_time + slew_time
            schedule_entry['exposure_time'] = exposure_time

            # we further get altitude and azimuth to check observability and add it to schedule entry list as well
            target_AltAz = target["SKY_COORD"].transform_to(AltAz(obstime = current_time, location = Wendelstein_loc))
            schedule_entry['Alt'] = target_AltAz.alt.to_value()
            schedule_entry['Az'] = target_AltAz.az.to_value()

            # add this entry to the complete list
            schedule.append(schedule_entry)

            # update the current time
            # current_time += slew_time + exposure_time # wrong place for this

            # if the object is observable (now location check), we add the energy, otherwise we don't add energy
            if target_AltAz.alt.to_value() > 0 and target_AltAz.alt.to_value() < 90:         
                energy += target['P_GAL']*((ending_time - current_time)/(ending_time - starting_time))**alpha
            else:
                unobservable_time = ending_time + utc_offset
                energy -= abs(target['P_GAL']*((ending_time - unobservable_time)/(ending_time - starting_time))**alpha)
            
            # update the sky location
            current_RA = target['RA']
            current_dec = target['dec']
        
        # since we can't observe the object before sunrise, add a negative schedule entry to the schedule list
        else:
            schedule_entry = {}
            schedule_entry['observed'] = False
            schedule.append(schedule_entry)

    # return the energy of this particular schedule and the schedule itself     
    return energy,schedule


### Simulated Annealing iteration and plan acceptance

This is the part where we iterate through many schedules and decide which to use as explained at the beginning.

In [9]:
# define initial sim annealing parameters
temperature  = 100
cooling_rate = 0.95
iterations   = 1000

# copy the initial observing plan and display it to see what changes have been introduced
current_observing_plan = initial_observing_plan.copy()
print("initial plan evaluation")
print(current_observing_plan)

# evaluate the first modified plan
current_energy, current_schedule = evaluate_plan(current_observing_plan, starting_time, ending_time)

# now iterate as often as specified above and change plans, evaluate and accept/not accept the plan
for i in tqdm(range(iterations)):
    #print("iteration ",i)
    
    # get a new plan and evaluate it
    proposed_observing_plan = change_plan(current_observing_plan) # makes a random change
    proposed_energy,proposed_schedule = evaluate_plan(proposed_observing_plan, starting_time, ending_time)
    
    # should we accept the new schedule?
    acceptance_probability = np.exp(-(current_energy-proposed_energy)/temperature) # >1 if proposed_energy>current_energy - always accept an improvement
    if(np.random.uniform(0,1)<acceptance_probability): # accept
        #print("accepting new plan")
        #print(f"acceptance_probability {acceptance_probability}, energy {current_energy}, proposed energy {proposed_energy} and temperature {temperature}")

        # if we accept the schedule, we update our variables accordingly
        current_observing_plan = proposed_observing_plan
        current_energy = proposed_energy
        current_schedule = proposed_schedule
        #print(current_observing_plan, current_schedule)
        
    temperature = temperature*cooling_rate

#print(f"acceptance_probability {acceptance_probability}, energy {current_energy}, proposed energy {proposed_energy} and temperature {temperature}")

# print the final observing plan and its schedule
print_observing_plan(current_observing_plan, current_schedule)

initial plan evaluation
             RA        dec    SERSIC         Z     P_GAL  \
0    175.636883  26.488867  4.938072  0.030302  0.001461   
1    169.094640  29.252292  3.370923  0.045417  0.001277   
2    168.792256  29.530045  4.509808  0.049634  0.001167   
3    149.470402  36.349205  5.137056  0.040875  0.001084   
4    168.013930  27.589772  4.207209  0.035104  0.001076   
..          ...        ...       ...       ...       ...   
195  126.676484  12.017623  4.863756  0.044725  0.000270   
196  148.824051  35.021667  1.540031  0.050197  0.000269   
197  129.366872  35.163570  2.193305  0.063617  0.000268   
198  177.243607  24.256910  4.100000  0.046490  0.000268   
199  143.042771  36.827124  5.871589  0.052932  0.000266   

                                             SKY_COORD  
0    <SkyCoord (ICRS): (ra, dec) in deg\n    (175.6...  
1    <SkyCoord (ICRS): (ra, dec) in deg\n    (169.0...  
2    <SkyCoord (ICRS): (ra, dec) in deg\n    (168.7...  
3    <SkyCoord (ICRS): (ra,

100%|██████████| 1000/1000 [01:51<00:00,  9.00it/s]

Our observation plan is 
              RA        dec    SERSIC         Z     P_GAL  \
0    189.900177  12.438881  5.110245  0.040751  0.000356   
1    170.369233  28.241693  4.000000  0.047731  0.000425   
2    168.180702  28.282739  3.036556  0.034535  0.000274   
3    142.381410  35.362824  4.748385  0.044928  0.000339   
4    156.193721  34.648938  4.072171  0.037290  0.000851   
..          ...        ...       ...       ...       ...   
195  182.690374  19.040828  4.900000  0.024982  0.000609   
196  177.121930  24.264487  4.640116  0.033944  0.000298   
197  195.920225   2.568118  4.626070  0.040708  0.000332   
198  150.721673  37.002905  6.000000  0.040818  0.000371   
199  157.182941  37.091817  3.220357  0.045557  0.000305   

                                             SKY_COORD  
0    <SkyCoord (ICRS): (ra, dec) in deg\n    (189.9...  
1    <SkyCoord (ICRS): (ra, dec) in deg\n    (170.3...  
2    <SkyCoord (ICRS): (ra, dec) in deg\n    (168.1...  
3    <SkyCoord (ICRS): (r

In [10]:
ra = 175.636883
dec = 26.488867 

from astropy import units as u
from astropy.coordinates import Angle

def print_3kk_command(ra, dec, eventname, bfilter, rfilter, nirfilter, texp, numobs):
    
    if(dec<0):
        print("not sure I am handling negative dec right")
    
    dec_a =  Angle(dec, u.degree)
    ra_a  =  Angle(ra, u.degree)
    print("2mobj ligodesi_" + superevent_name + "_" + "{:02d}".format(int(ra_a.hms.h)) + "{:02d}".format(int(ra_a.hms.m)) + "{:02d}".format(int(ra_a.hms.s))
                                        + "_" + "{:02d}".format(int(dec_a.dms.d)) + "{:02d}".format(int(dec_a.dms.m)) + "{:02d}".format(int(dec_a.dms.s)) + " 3441")
    ra_a = ra_a + Angle(35,u.arcsec)
    print("2mradec " + str(int(ra_a.hms.h)) + " " + str(int(ra_a.hms.m)) + " " + "{:2.2f}".format(ra_a.hms.s) + "   " + str(int(dec_a.dms.d)) + " " + str(int(dec_a.dms.m)) + " " + "{:2.2f}".format(dec_a.dms.s) )
    print(f"2mf b{bfilter} r{rfilter} n{nirfilter}")
    
    print("2mdithersize 10")
    print("2mh best")
    print("2mh calcz")
    print("2mautofocus")
    print("2mt " + str(int(texp))) 
    print("2mo " + str(int(numobs)))
    print(f"#open https://www.legacysurvey.org/viewer/jpeg-cutout?ra={round(ra_a.degree,4)}&dec={round(dec_a.degree,4)}&layer=ls-dr10&pixscale=0.262")
    print(f"#wget https://www.legacysurvey.org/viewer/fits-cutout?ra={round(ra_a.degree,4)}&dec={round(dec_a.degree,4)}&layer=ls-dr10&pixscale=0.262&bands=griz")
    

# print_3kk_command(ra = ra, dec = dec, eventname = 'S230615az', bfilter =  , rfilter = "i", nirfilter= "J", texp = 100, numobs = 5)

In [11]:
# For targets 230516

GOTO23hn = SkyCoord('12:40:35.55 +17:02:15.27', unit=(u.hourangle, u.deg))
GOTO23hu = SkyCoord('13:22:55.23 +08:09:49.57', unit=(u.hourangle, u.deg))
superevent_name = 'GOTO23hn'
print_3kk_command(ra = GOTO23hn.ra, dec = GOTO23hn.dec, eventname = 'GOTO23hn', bfilter =  "g", rfilter = "z", nirfilter= "J", texp = 700, numobs = 3)

print_3kk_command(ra = GOTO23hu.ra, dec = GOTO23hu.dec, eventname = 'GOTO23hu', bfilter =  "g", rfilter = "z", nirfilter= "J", texp = 700, numobs = 3)

2mobj ligodesi_GOTO23hn_124035_170215 3441
2mradec 12 40 37.88   17 2 15.27
2mf bg rz nJ
2mdithersize 10
2mh best
2mh calcz
2mautofocus
2mt 700
2mo 3
#open https://www.legacysurvey.org/viewer/jpeg-cutout?ra=190.1578&dec=17.0376&layer=ls-dr10&pixscale=0.262
#wget https://www.legacysurvey.org/viewer/fits-cutout?ra=190.1578&dec=17.0376&layer=ls-dr10&pixscale=0.262&bands=griz
2mobj ligodesi_GOTO23hn_132255_080949 3441
2mradec 13 22 57.56   8 9 49.57
2mf bg rz nJ
2mdithersize 10
2mh best
2mh calcz
2mautofocus
2mt 700
2mo 3
#open https://www.legacysurvey.org/viewer/jpeg-cutout?ra=200.7398&dec=8.1638&layer=ls-dr10&pixscale=0.262
#wget https://www.legacysurvey.org/viewer/fits-cutout?ra=200.7398&dec=8.1638&layer=ls-dr10&pixscale=0.262&bands=griz


In [12]:
# print(target_list)

ra = target_list['RA'][:10]
dec = target_list['dec'][:10]

# for i in range(len(ra)):
    # print_3kk_command(ra = ra[i], dec = dec[i], eventname = 'S230615az', bfilter =  "r", rfilter = "i", nirfilter= "J", texp = 100, numobs = 5)
# for ra_i, dec_i in ra, dec:
#     print_3kk_command(ra = ra, dec = dec, eventname = 'S230615az', bfilter =  "r", rfilter = "i", nirfilter= "J", texp = 100, numobs = 5)

In [13]:
# For the GROWTH candidates

# ra_growth = ['12:50:03.641','12:42:11.651']
# dec_growth = ['+20:53:21.77', '+22:03:25.08']

growth_1 = SkyCoord('12:50:03.641 +20:53:21.77', unit=(u.hourangle, u.deg))
print(growth_1)
growth_2 = SkyCoord('12:42:11.651 +22:03:25.08', unit=(u.hourangle, u.deg))
print_3kk_command(ra = growth_1.ra, dec = growth_1.dec, eventname = 'S230615az', bfilter =  "r", rfilter = "z", nirfilter= "K", texp = 50, numobs = 3)

print_3kk_command(ra = growth_2.ra, dec = growth_2.dec, eventname = 'S230615az', bfilter =  "r", rfilter = "z", nirfilter= "K", texp = 50, numobs = 3)

<SkyCoord (ICRS): (ra, dec) in deg
    (192.51517083, 20.88938056)>
2mobj ligodesi_GOTO23hn_125003_205321 3441
2mradec 12 50 5.97   20 53 21.77
2mf br rz nK
2mdithersize 10
2mh best
2mh calcz
2mautofocus
2mt 50
2mo 3
#open https://www.legacysurvey.org/viewer/jpeg-cutout?ra=192.5249&dec=20.8894&layer=ls-dr10&pixscale=0.262
#wget https://www.legacysurvey.org/viewer/fits-cutout?ra=192.5249&dec=20.8894&layer=ls-dr10&pixscale=0.262&bands=griz
2mobj ligodesi_GOTO23hn_124211_220325 3441
2mradec 12 42 13.98   22 3 25.08
2mf br rz nK
2mdithersize 10
2mh best
2mh calcz
2mautofocus
2mt 50
2mo 3
#open https://www.legacysurvey.org/viewer/jpeg-cutout?ra=190.5583&dec=22.057&layer=ls-dr10&pixscale=0.262
#wget https://www.legacysurvey.org/viewer/fits-cutout?ra=190.5583&dec=22.057&layer=ls-dr10&pixscale=0.262&bands=griz


In [14]:
# Calculate Alt, Az from RA Dec

ra = [GOTO23hn.ra, GOTO23hu.ra]
dec = [GOTO23hn.dec, GOTO23hu.dec]

dec = target_list['dec']
times = ['2023-06-16T22:00']
t = Time(times, format='isot', scale='utc')
WST = EarthLocation(lat=47.703625*u.deg, lon=12.012669*u.deg, height=1837.*u.m)
# objects = SkyCoord(ra[11], dec[11], unit = 'degree')
# Create Sky Coord instances for all 
for i in range(len(ra)):
    object = SkyCoord(ra[i], dec[i], unit = 'degree')
    altaz = object.transform_to(AltAz(obstime=times, location=WST))
    if altaz.az.deg>10:
        print(f"Altitude at time {times} is {altaz.alt}, Azimuth is {altaz.az}")

Altitude at time ['2023-06-16T22:00'] is [41.40630847] deg, Azimuth is [264.86720937] deg
Altitude at time ['2023-06-16T22:00'] is [50.26948636] deg, Azimuth is [259.00769059] deg


## Download the first N_download image fits files from the LS survey for difference imaging

In [15]:
import requests as rq
import os

from astropy.table import Table, hstack
from astropy.io import fits


N_download = 10

FOV = 7*1/60 # FOV for 3kk, for WWFI change the 7 to a 30
             # note: the FOV of WWFI (as the name "wide-field-imager-suggests") is so large that (assuming pixscale 0.262) we need ~8000 pixels and therefore the LS fits image is ~130mb large

pixscale = 0.262*1/3600 #native pixscale from LS: https://www.legacysurvey.org/dr10/description/
size = int(1.2*FOV/pixscale)

print(f"For a FOV of {FOV*u.deg} and a pixel scale from LS of {pixscale*u.arcsec} we need an image of size {size} pixel to cover 1.2 times the FOV (i.e. {1.2*FOV*u.deg})")


For a FOV of 0.11666666666666667 deg and a pixel scale from LS of 7.277777777777778e-05 arcsec we need an image of size 1923 pixel to cover 1.2 times the FOV (i.e. 0.13999999999999999 deg)


In [16]:
obs_plan = Table.from_pandas(current_observing_plan)

In [17]:
newpath = r'/global/homes/j/jgassert/ligo-desi-gw-follow-up/Galaxy Selection/Follow-Up-Images/'
newpath = newpath +superevent_name + "/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

PermissionError: [Errno 13] Permission denied: '/global/homes/j/jgassert/ligo-desi-gw-follow-up/Galaxy Selection'

In [18]:
for target in tqdm(obs_plan[:1]):
    #print(target)
    RA, dec = target["RA"], target["dec"]
    download_url = f"https://www.legacysurvey.org/viewer/fits-cutout?ra={RA}&dec={dec}&layer=ls-dr10&pixscale={pixscale}&bands=griz&size={size}"
    #save_path = f"Follow-Up-Images/{superevent_name}_{RA}_{dec}.fits"
    save_path = "testfile.fits"
    
    response = rq.get(download_url, allow_redirects = True)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
    else:
        print(f"download image fits file for object at RA {RA} and dec {dec} failed")

100%|██████████| 1/1 [00:04<00:00,  4.03s/it]


In [19]:
# Function to print out terminal commands for Difference Imaging

superevent_name = 'S230615az'
def print_diffimg_commands(ra, dec, eventname, filter):
    ra = target_list['RA'][:10]
    dec = target_list['dec'][:10]
    eventname = superevent_name
    CCD = {'i':1, 'r': 0}
    # CCD_r = 0
    ra_a = Angle(ra, u.degree)
    # print(ra_a.deg)
    dec_a =  Angle(dec, u.degree)
    # print(ra_a.hms,dec_a.hms)
    # print(dec_a)
    num = ''
    for x in superevent_name:
        if x.isdigit():
            num += str(x)
    for i in range(len(ra)):
        print("\n"+ "WORKDIR=/data/wst/u/ps1ipp/arri/3kk")
        print("mkdir $WORKDIR/bias")
        print("mkdir $WORKDIR/dark")
        print("mkdir $WORKDIR/flat")
        print("cd $WORKDIR")
        print(f"/data/wst/u/ps1ipp/arri/3kk/script_3kk_ligodesi.sh  ligodesi_{superevent_name}" + "_" + "{:02d}".format(int(ra_a[i].hms.h)) + "{:02d}".format(int(ra_a[i].hms.m)) + "{:02d}".format(int(ra_a[i].hms.s))+ "_" + "{:02d}".format(int(dec_a[i].dms.d)) + "{:02d}".format(int(dec_a[i].dms.m)) + "{:02d}".format(int(dec_a[i].dms.s)) + '  ' +"20"+ num + " " + f"{filter}")
        print(f"cd ligodesi_{superevent_name}" + "_" + "{:02d}".format(int(ra_a[i].hms.h)) + "{:02d}".format(int(ra_a[i].hms.m)) + "{:02d}".format(int(ra_a[i].hms.s)) + "_" + "{:02d}".format(int(dec_a[i].dms.d)) + "{:02d}".format(int(dec_a[i].dms.m)) + "{:02d}".format(int(dec_a[i].dms.s)) + "/" + "20" + num + "_" + f"{filter}")
        print(f"/data/wst/u/ps1ipp/arri/3kk/script_3kk_header_for_astrometry.sh  RA={ra_a[i].deg}  DEC={dec_a[i].deg}  CCD={CCD[filter]}  mepdnybo-n2023*.fits")
        print(f"sethead  FILTER={filter} mepdnybo-n*.fits")
        print("for i in mepdnybo-*.fits; do  script.astrometry.sh $i 1;  done")
        print("FIRST=`echo vme*.fits | awk '{print $1}'`")
        print("divvy  -e          -2  -3  -wcsok  -ref ${FIRST}    vme*.fits ")
        print("sumfits -n tvvmepdnybo-n2023*.fits")
print_diffimg_commands(target_list['RA'], target_list['dec'], 'S230615az', 'i')


# superevent_name = 'S230615az'
# # num = [int(x) for x in superevent_name.split() if x.isdigit()]
# output = print_diffimg_commands(target_list['RA'], target_list['dec'], 'S230615az', 'r')
# filename= open(f"{superevent_name}_{filter}.txt","w+")
# filename.write(output)
# filename.close()




WORKDIR=/data/wst/u/ps1ipp/arri/3kk
mkdir $WORKDIR/bias
mkdir $WORKDIR/dark
mkdir $WORKDIR/flat
cd $WORKDIR
/data/wst/u/ps1ipp/arri/3kk/script_3kk_ligodesi.sh  ligodesi_S230615az_114232_262919  20230615 i
cd ligodesi_S230615az_114232_262919/20230615_i
/data/wst/u/ps1ipp/arri/3kk/script_3kk_header_for_astrometry.sh  RA=175.63688326451  DEC=26.4888665596758  CCD=1  mepdnybo-n2023*.fits
sethead  FILTER=i mepdnybo-n*.fits
for i in mepdnybo-*.fits; do  script.astrometry.sh $i 1;  done
FIRST=`echo vme*.fits | awk '{print $1}'`
divvy  -e          -2  -3  -wcsok  -ref ${FIRST}    vme*.fits 
sumfits -n tvvmepdnybo-n2023*.fits

WORKDIR=/data/wst/u/ps1ipp/arri/3kk
mkdir $WORKDIR/bias
mkdir $WORKDIR/dark
mkdir $WORKDIR/flat
cd $WORKDIR
/data/wst/u/ps1ipp/arri/3kk/script_3kk_ligodesi.sh  ligodesi_S230615az_111622_291508  20230615 i
cd ligodesi_S230615az_111622_291508/20230615_i
/data/wst/u/ps1ipp/arri/3kk/script_3kk_header_for_astrometry.sh  RA=169.094639962215  DEC=29.2522918090561  CCD=1  mepdny

In [ ]:
# Growth Candidates

growth_1 = SkyCoord('12:50:03.641 +20:53:21.77', unit=(u.hourangle, u.deg))
growth_2 = SkyCoord('12:42:11.651 +22:03:25.08', unit=(u.hourangle, u.deg))

